In [13]:
# pip install mlxtend

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.1
    Uninstalling scikit-learn-0.23.1:
      Successfully uninstalled scikit-learn-0.23.1
Note: you may need to restart the kernel to use updated packages.


ERROR: scikit-learn 1.2.0 has requirement joblib>=1.1.1, but you'll have joblib 0.16.0 which is incompatible.


In [40]:
from csv import reader
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules
import numpy as np


In [10]:
groceries = []

with open('data/groceries.csv', 'r', encoding='utf-8-sig') as csvfile:
    csv_reader = reader(csvfile)
    for row in csv_reader:
        groceries.append(row)

In [12]:
groceries[0:5]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

In [17]:
# Instantiate an encoder object
encoder = TransactionEncoder()
# Use the fit method to extract unique labels in the set
# Use the transform method to one-hot encode the transactions into a boolean
transactions = encoder.fit(groceries).transform(groceries)
transactions

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [19]:
# Convert the transactions to a dataframe
itemsets = pd.DataFrame(transactions, columns = encoder.columns_)
itemsets.head()

,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,...,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,sound storage medium,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,uht-milk,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False


In [20]:
# Summary of the dataframe
itemsets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9835 entries, 0 to 9834
Columns: 169 entries, abrasive cleaner to zwieback
dtypes: bool(169)
memory usage: 1.6 MB


# Generate frequent itemsets

Assume we want itemsets that occur at least 5 times a day. The data spans 30 days and has 9,835 transactions. We need to set our minimum support threshold to $5 \times \frac{30}{9835} \approx 0.015$

In [23]:
# Display the frequent itemsets with column names
frequent_itemsets = apriori(itemsets, min_support = 0.015, use_colnames = True)
frequent_itemsets

,support,itemsets
0,0.017692,(baking powder)
1,0.052466,(beef)
2,0.033249,(berries)
3,0.026029,(beverages)
4,0.080529,(bottled beer)
...,...,...
175,0.023183,"(other vegetables, root vegetables, whole milk)"
176,0.017082,"(other vegetables, whole milk, tropical fruit)"
177,0.022267,"(other vegetables, yogurt, whole milk)"
178,0.015557,"(yogurt, whole milk, rolls/buns)"


In [25]:
# Sort by support
frequent_itemsets.sort_values('support', ascending = False)

,support,itemsets
71,0.255516,(whole milk)
44,0.193493,(other vegetables)
53,0.183935,(rolls/buns)
60,0.174377,(soda)
72,0.139502,(yogurt)
...,...,...
163,0.015252,"(shopping bags, yogurt)"
179,0.015150,"(yogurt, whole milk, tropical fruit)"
45,0.015048,(pasta)
11,0.015048,(canned fish)


In [26]:
# Let's look at only frequent itemsets with a length greater than two. First get the length of the itemsets:
length = frequent_itemsets['itemsets'].str.len()
# Then create a filter:
rows = length > 2
# Finally, apply the filter to the dataframe:
frequent_itemsets[rows]

,support,itemsets
174,0.017895,"(other vegetables, whole milk, rolls/buns)"
175,0.023183,"(other vegetables, root vegetables, whole milk)"
176,0.017082,"(other vegetables, whole milk, tropical fruit)"
177,0.022267,"(other vegetables, yogurt, whole milk)"
178,0.015557,"(yogurt, whole milk, rolls/buns)"
179,0.015150,"(yogurt, whole milk, tropical fruit)"


In [27]:
frequent_itemsets.groupby(length)['support'].describe()

,count,mean,std,min,25%,50%,75%,max
itemsets,,,,,,,,
1,73.0,0.053441,0.045956,0.015048,0.024504,0.037112,0.064870,0.255516
2,101.0,0.024799,0.010058,0.015048,0.018404,0.021047,0.027555,0.074835
3,6.0,0.018522,0.003417,0.015150,0.015938,0.017489,0.021174,0.023183


Most transactions considered frequent are two-item purchases (count: 101). Single-item purchases have a higher support value (according to the mean and 50% columns) than two- and three-item purchases.

# Create association rules

In [30]:
# Generate rules - looking for confidence with a minimum threshold of 0.25
rules = association_rules(frequent_itemsets, metric = 'confidence', min_threshold = 0.25)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(beef),(other vegetables),0.052466,0.193493,0.019725,0.375969,1.943066,0.009574,1.292416
1,(beef),(root vegetables),0.052466,0.108998,0.017387,0.331395,3.040367,0.011668,1.332628
2,(beef),(whole milk),0.052466,0.255516,0.021251,0.405039,1.585180,0.007845,1.251315
3,(bottled beer),(whole milk),0.080529,0.255516,0.020437,0.253788,0.993237,-0.000139,0.997684
4,(bottled water),(soda),0.110524,0.174377,0.028978,0.262190,1.503577,0.009705,1.119017
...,...,...,...,...,...,...,...,...,...
73,"(yogurt, rolls/buns)",(whole milk),0.034367,0.255516,0.015557,0.452663,1.771563,0.006775,1.360192
74,"(whole milk, rolls/buns)",(yogurt),0.056634,0.139502,0.015557,0.274686,1.969049,0.007656,1.186380
75,"(yogurt, whole milk)",(tropical fruit),0.056024,0.104931,0.015150,0.270417,2.577089,0.009271,1.226823
76,"(yogurt, tropical fruit)",(whole milk),0.029283,0.255516,0.015150,0.517361,2.024770,0.007668,1.542528


In [31]:
# Filter to see only 'rolls/buns' in the antecedent
rows = rules['antecedents'] == {'rolls/buns'}
# Apply the filter
rules[rows]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
51,(rolls/buns),(whole milk),0.183935,0.255516,0.056634,0.307905,1.205032,0.009636,1.075696


In [32]:
# Repeat the process for the consequent
rows = rules['consequents'] == {'rolls/buns'}
rules[rows]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
24,(frankfurter),(rolls/buns),0.058973,0.183935,0.019217,0.325862,1.771616,0.008370,1.210531
50,(sausage),(rolls/buns),0.093950,0.183935,0.030605,0.325758,1.771048,0.013324,1.210344
72,"(yogurt, whole milk)",(rolls/buns),0.056024,0.183935,0.015557,0.277677,1.509648,0.005252,1.129779


In [33]:
# Filter to see all rules with 'rolls/buns' as either the antecedent or consequent
rows = rules['antecedents'].astype(str).str.contains('rolls/buns')
rules[rows]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
51,(rolls/buns),(whole milk),0.183935,0.255516,0.056634,0.307905,1.205032,0.009636,1.075696
62,"(other vegetables, rolls/buns)",(whole milk),0.042603,0.255516,0.017895,0.420048,1.643919,0.007010,1.283699
63,"(whole milk, rolls/buns)",(other vegetables),0.056634,0.193493,0.017895,0.315978,1.633026,0.006937,1.179067
73,"(yogurt, rolls/buns)",(whole milk),0.034367,0.255516,0.015557,0.452663,1.771563,0.006775,1.360192
74,"(whole milk, rolls/buns)",(yogurt),0.056634,0.139502,0.015557,0.274686,1.969049,0.007656,1.186380


In [34]:
# Filter for rules with a length of 2 or more
rows = rules['antecedents'].str.len() > 1
rules[rows]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
62,"(other vegetables, rolls/buns)",(whole milk),0.042603,0.255516,0.017895,0.420048,1.643919,0.007010,1.283699
63,"(whole milk, rolls/buns)",(other vegetables),0.056634,0.193493,0.017895,0.315978,1.633026,0.006937,1.179067
64,"(other vegetables, root vegetables)",(whole milk),0.047382,0.255516,0.023183,0.489270,1.914833,0.011076,1.457687
65,"(other vegetables, whole milk)",(root vegetables),0.074835,0.108998,0.023183,0.309783,2.842082,0.015026,1.290900
66,"(root vegetables, whole milk)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320
67,"(other vegetables, tropical fruit)",(whole milk),0.035892,0.255516,0.017082,0.475921,1.862587,0.007911,1.420556
68,"(whole milk, tropical fruit)",(other vegetables),0.042298,0.193493,0.017082,0.403846,2.087140,0.008898,1.352851
69,"(other vegetables, yogurt)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340
70,"(other vegetables, whole milk)",(yogurt),0.074835,0.139502,0.022267,0.297554,2.132979,0.011828,1.225003
71,"(yogurt, whole milk)",(other vegetables),0.056024,0.193493,0.022267,0.397459,2.054131,0.011427,1.338511


In [35]:
# Let's look for a lift of more than 2, leverage score more than 0.01, and a conviction score of more than 1.4
rows = ((rules['lift'] > 2) & (rules['leverage'] > 0.01) & (rules['conviction'] > 1.4))
rules[rows]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
66,"(root vegetables, whole milk)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320
69,"(other vegetables, yogurt)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340


# Evaluate association rules

In [36]:
rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,0.073186,0.211041,0.025578,0.360834,1.763158,0.010156,1.245113
std,0.036738,0.046866,0.012045,0.070495,0.377079,0.004766,0.115688
min,0.029283,0.104931,0.015048,0.253714,0.993237,-0.000139,0.997684
25%,0.052466,0.193493,0.017895,0.308374,1.504669,0.006970,1.166832
50%,0.061210,0.193493,0.021912,0.354567,1.740032,0.009260,1.226608
75%,0.082766,0.255516,0.028876,0.405608,1.942669,0.011788,1.294081
max,0.255516,0.255516,0.074835,0.517361,3.040367,0.026291,1.542528


In [37]:
# Explore the top 5 rules by lift
rules.sort_values('lift', ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(beef),(root vegetables),0.052466,0.108998,0.017387,0.331395,3.040367,0.011668,1.332628
65,"(other vegetables, whole milk)",(root vegetables),0.074835,0.108998,0.023183,0.309783,2.842082,0.015026,1.290900
75,"(yogurt, whole milk)",(tropical fruit),0.056024,0.104931,0.015150,0.270417,2.577089,0.009271,1.226823
47,(pip fruit),(tropical fruit),0.075648,0.104931,0.020437,0.270161,2.574648,0.012499,1.226392
77,"(whole milk, tropical fruit)",(yogurt),0.042298,0.139502,0.015150,0.358173,2.567516,0.009249,1.340701


In [38]:
# The lift values above are above 1, meaning we consider it a factor of likelihood (the first rule shows that customers who bought beef are 3.04 times more likely to purchase root vegetables).

# Explore the top 5 rules by leverage
rules.sort_values('leverage', ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39,(root vegetables),(other vegetables),0.108998,0.193493,0.047382,0.434701,2.246605,0.026291,1.426693
43,(other vegetables),(whole milk),0.193493,0.255516,0.074835,0.386758,1.513634,0.025394,1.214013
44,(whole milk),(other vegetables),0.255516,0.193493,0.074835,0.292877,1.513634,0.025394,1.140548
52,(root vegetables),(whole milk),0.108998,0.255516,0.048907,0.448694,1.756031,0.021056,1.350401
61,(yogurt),(whole milk),0.139502,0.255516,0.056024,0.401603,1.571735,0.020379,1.244132


In [39]:
# These rules seem to be in conflict with each other (or at least deceiving). Maybe the consequent and antecedent are not dependent on each other?

# Let's explore the top 5 rules by conviction
rules.sort_values('conviction', ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
76,"(yogurt, tropical fruit)",(whole milk),0.029283,0.255516,0.015150,0.517361,2.024770,0.007668,1.542528
66,"(root vegetables, whole milk)",(other vegetables),0.048907,0.193493,0.023183,0.474012,2.449770,0.013719,1.533320
69,"(other vegetables, yogurt)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,0.011174,1.528340
9,(butter),(whole milk),0.055414,0.255516,0.027555,0.497248,1.946053,0.013395,1.480817
19,(curd),(whole milk),0.053279,0.255516,0.026131,0.490458,1.919481,0.012517,1.461085


In [43]:
# Calculate Zhang's metric and add it to the rules dataframe

def zhang_metric(rules):
    sup = rules['support'].copy()
    sup_a = rules['antecedent support'].copy()
    sup_b = rules['consequent support'].copy()
    num = sup - sup_a * sup_b
    denom = np.max((sup * (1 - sup_a).values, sup_a * (sup_b - sup).values), axis = 0)
    return num / denom

In [44]:
# Assign the function output to a new column
rules['zhang'] = zhang_metric(rules)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
0,(beef),(other vegetables),0.052466,0.193493,0.019725,0.375969,1.943066,0.009574,1.292416,0.512224
1,(beef),(root vegetables),0.052466,0.108998,0.017387,0.331395,3.040367,0.011668,1.332628,0.708251
2,(beef),(whole milk),0.052466,0.255516,0.021251,0.405039,1.585180,0.007845,1.251315,0.389597
3,(bottled beer),(whole milk),0.080529,0.255516,0.020437,0.253788,0.993237,-0.000139,0.997684,-0.007351
4,(bottled water),(soda),0.110524,0.174377,0.028978,0.262190,1.503577,0.009705,1.119017,0.376535
...,...,...,...,...,...,...,...,...,...,...
73,"(yogurt, rolls/buns)",(whole milk),0.034367,0.255516,0.015557,0.452663,1.771563,0.006775,1.360192,0.451027
74,"(whole milk, rolls/buns)",(yogurt),0.056634,0.139502,0.015557,0.274686,1.969049,0.007656,1.186380,0.521686
75,"(yogurt, whole milk)",(tropical fruit),0.056024,0.104931,0.015150,0.270417,2.577089,0.009271,1.226823,0.648285
76,"(yogurt, tropical fruit)",(whole milk),0.029283,0.255516,0.015150,0.517361,2.024770,0.007668,1.542528,0.521384


In [45]:
# Consider the top 5 by Zhang metric
rules.sort_values('zhang', ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
1,(beef),(root vegetables),0.052466,0.108998,0.017387,0.331395,3.040367,0.011668,1.332628,0.708251
65,"(other vegetables, whole milk)",(root vegetables),0.074835,0.108998,0.023183,0.309783,2.842082,0.015026,1.290900,0.700572
47,(pip fruit),(tropical fruit),0.075648,0.104931,0.020437,0.270161,2.574648,0.012499,1.226392,0.661650
75,"(yogurt, whole milk)",(tropical fruit),0.056024,0.104931,0.015150,0.270417,2.577089,0.009271,1.226823,0.648285
77,"(whole milk, tropical fruit)",(yogurt),0.042298,0.139502,0.015150,0.358173,2.567516,0.009249,1.340701,0.637483


In [46]:
# The above have a strong positive association. 

# Consider the bottom 5 by Zhang metric
rules.sort_values('zhang', ascending = True).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
3,(bottled beer),(whole milk),0.080529,0.255516,0.020437,0.253788,0.993237,-0.000139,0.997684,-0.007351
5,(bottled water),(whole milk),0.110524,0.255516,0.034367,0.310948,1.216940,0.006126,1.080446,0.200417
51,(rolls/buns),(whole milk),0.183935,0.255516,0.056634,0.307905,1.205032,0.009636,1.075696,0.208496
54,(sausage),(whole milk),0.093950,0.255516,0.029893,0.318182,1.245252,0.005887,1.091910,0.217372
16,(coffee),(whole milk),0.058058,0.255516,0.018709,0.322242,1.261141,0.003874,1.098451,0.219830


In [ ]:
# This first rule has a negative score, indicating a slight dissociation. Meaning, moving bottled beer and whole milk away from each other would not have a significant impact, nor would it be a good idea to pair them together for a promotional sale.